In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

dest_path = dbutils.widgets.get("dest_path")

# List all result files
result_files = dbutils.fs.ls("tmp/")
text_analysis_files = [f.path for f in result_files if f.name.endswith("_text_analysis.csv")]

# Define schema explicitly to ensure correct data types
schema = StructType([
    StructField("file_name", StringType(), True),
    StructField("raw_text", StringType(), True),
    StructField("total_word_count", IntegerType(), True),
    StructField("sentiment", StringType(), True)
])

# Read each CSV with proper options to handle quoted fields and multiline text
df_list = []
for path in text_analysis_files:
    try:
        df = spark.read.format("csv") \
            .option("header", "true") \
            .option("multiline", "true") \
            .option("quote", "\"") \
            .option("escape", "\"") \
            .schema(schema) \
            .load(path)
        
        # Show a sample to verify data is loaded correctly
        df.select("file_name", "total_word_count", "sentiment").show(1, truncate=False)
        
        df_list.append(df)
    except Exception as e:
        print(f"Error reading {path}: {str(e)}")

# If there are no files, create an empty dataframe with the right schema
if not df_list:
    print("No valid files found, creating empty dataframe")
    all_results = spark.createDataFrame([], schema)
else:
    # Union all the outputs
    all_results = df_list[0]
    for df in df_list[1:]:
        all_results = all_results.union(df)

# No need to cast columns as we defined the schema explicitly
final_df = all_results

# Print the schema and a sample of data for debugging
final_df.printSchema()

final_df.select("file_name", "total_word_count", "sentiment").show(5, truncate=False)

final_df.write.format("delta").mode("overwrite").saveAsTable(dest_path)